In [ ]:
'''
'''

'''
is it necessary to convert trade_date to to_date?

No, it’s not strictly necessary to convert trade_date to a proper DateType if you only need to calculate the daily price change and order by the date in the format yyyy-MM-dd.

Here’s why:
String Ordering Works:
The date strings in yyyy-MM-dd format naturally sort correctly in chronological order when used in ORDER BY.
So LAG() or any window function will still work as intended.

When to_date is Useful:

If you want to perform date arithmetic, e.g., calculating the number of days between two dates.
If your date is not in yyyy-MM-dd format.

For date functions like month(), year(), dayofweek(), etc.
'''

